In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import tensorflow as tf
from tensorflow.keras import Input, Model
from keras.layers import Dense, Dropout, Flatten, add
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import keras
from tensorflow.keras.applications import DenseNet201
from keras.applications.mobilenet import MobileNet
from keras.applications.densenet import DenseNet169
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomCrop, Rescaling, RandomTranslation
from keras import Sequential
from tqdm import tqdm

In [ ]:
root_dir = '../input/cassava-leaf-disease-classification'

train_df = pd.read_csv(os.path.join(root_dir, 'train.csv'))
print("there are " + str(train_df.shape[0]) + " train samples" )
train_df.head()

In [ ]:
train_df.info()

In [ ]:

train_img_dir = os.path.join(root_dir, 'train_images')  

figure = plt.figure(figsize = (20,20))

cont = 0
    
for i in range(5):
    
    speci = train_df[train_df['label'] == i]
    
    for j in range(5):
        
        img = Image.open(os.path.join(train_img_dir, speci.iloc[j,0]))
        
        plt.subplot(5,5, cont+1)
        
        plt.imshow(img)
        
        cont = cont + 1

In [ ]:

image_preprocessor = Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomCrop(300,300),
    RandomTranslation(0.3, 0.3),
    RandomRotation(0.5),
    Rescaling(1./255)])

def custom_gen(batch_size, image_dir, h = 300, w = 300):
    
    start = 0
    end = batch_size
    images = train_df['image_id']
    labels = train_df['label']
    while 1:
        
        if end >= train_df.shape[0]:
            start = 0
            end = batch_size 
            continue
        else:
        
            batch = []

            if start == 0:
                names = images[:end]
                y = to_categorical(labels[:end], num_classes = 5)
            else:
                names = images[start:end]
                y = to_categorical(labels[start:end], num_classes = 5)

            for name in names:

                img = cv2.imread(os.path.join(image_dir,name))
                img = np.expand_dims(img, axis = 0)
                img = image_preprocessor(img)
                img = np.squeeze(img, axis = 0)
                batch.append(img)



            end = end + batch_size
            start = start +  batch_size


            yield np.array(batch), y
            
            
            
            

In [ ]:
#densenet169_dir = "../input/allmodelwthih5/densenet169_weights_tf_dim_ordering_tf_kernels_notop.h5"
#base_dens = DenseNet169(weights = densenet169_dir, include_top = False, input_shape = (300, 300, 3))
#base_dens.trainable = False

In [ ]:
#dens_base = create_base_model(base_dens)

In [ ]:
# MobileNet Base Model creating NON trainable
mobilenet_weights_dir = '../input/allmodelwthih5/mobilenet_1_0_224_tf_no_top.h5'
base_mob = MobileNet(weights = mobilenet_weights_dir, include_top = False)
base_mob.trainable  = False

vgg19_weights_dir = "../input/allmodelwthih5/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_vgg19 = VGG19(weights = vgg19_weights_dir, include_top = False, input_shape = (300,300,3))
base_vgg19.trainable = False


# function to create a base model with standard output of 1024

def create_base_model(base_model):
    
    inputs = Input(shape = (300,300,3))
    
    base_out = base_model(inputs)
    
    base_out = Flatten()(base_out)
    
    base_out = Dropout(0.3)(base_out)
    
    base1_out = Dense(1024, activation = 'relu')(base_out)
    
    model = Model(inputs = inputs, outputs = base1_out)
    
    return model


vgg19_base = create_base_model(base_vgg19)

mob_base = create_base_model(base_mob)





def create_model(base1, base2):
    
    inputs = Input(shape = (300,300,3))
    
    base1_out = base1(inputs)
    base2_out = base2(inputs)

    base_out = add([base1_out, base2_out])
    
    outputs = Dense(512, activation = 'relu')(base_out)
    
    outputs = Dropout(0.2)(outputs)
    
    outputs = Dense(5, activation = 'softmax')(outputs)
    
    model = Model(inputs = inputs, outputs = outputs)
    
    model.summary()

    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy')
    
    lrd = ReduceLROnPlateau(monitor = 'val_loss',
                         patience = 200,
                         verbose = 1,
                         factor = 0.75,
                         min_lr = 1e-10)

    mcp = ModelCheckpoint('model.h5')

    es = EarlyStopping(verbose=1, patience=60)
    
    return model
    
    
model = create_model(mob_base, vgg19_base)

In [ ]:
batch_size = 64
epochs = 25
steps_per_epoch = train_df.shape[0] // batch_size
train_gen = custom_gen(batch_size, train_img_dir)

history = model.fit(train_gen, epochs = epochs, steps_per_epoch = steps_per_epoch )

In [ ]:
history.history.keys()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



In [ ]:
test_leaf = "../input/cassava-leaf-disease-classification/test_images"

test_names = pd.Series(os.listdir(test_leaf))


for j in range(3):

    for i in tqdm(range(len(test_names))):

        image = cv2.imread(os.path.join(test_leaf, test_names[i]))
        image = np.expand_dims(image, axis = 0)
        image = image_preprocessor(image)
        if i ==0:

            pred = model.predict(image)
        else:
            pred = np.concatenate([pred, model.predict(image)])
            
    if j ==0:
        final = pred
    else:
        final = final +pred
     
pred = pd.Series(np.argmax(final, axis = 1))


test_df = pd.concat([test_names, pred], axis = 1)
test_df = test_df.rename(columns = {0: 'image_id', 1: 'label'})

test_df.to_csv('submission.csv', index = False)